# <font color='gray'> Lab 5: Part-II -- ROC Curves</font>

In this exercise, we will explore the ROC curves as a means to evaluate classifiers, and the benefits they bring compared to simple accuracy metrics. 

#### 0. In the the following, we will set up a two-class classification problem based on the iris flower data you are already familiar with. As you can see the data is quite overlapped, so it will be hard to classify accurately (we have chosen the flower classes that are the most difficult to separate).

In [0]:
import matplotlib.pyplot as plt
from sklearn import datasets

# import some data to play with
iris = datasets.load_iris()
 # we only take the first two features, but also only the two difficult classes:
idx = (iris.target == 1)|(iris.target == 2)
X = iris.data[idx, :2] 
Y = iris.target[idx]

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)
ax.set_aspect('equal')

colors = ("green", "blue")
marker_list = ['o', 'x']


for l in [1, 2]:
  ax.scatter(X[Y == l, 0], X[Y == l, 1],
             marker=marker_list[l-1], s=70,
             c=colors[l-1], edgecolors='none',
             label='{:d} ({:s})'.format(l, iris.target_names[l]))

ax.legend(fontsize=12,loc='upper left')
ax.set_xlabel(iris.feature_names[0], fontsize=14)
ax.set_ylabel(iris.feature_names[1], fontsize=14)
ax.grid(alpha=0.3)
ax.set_xlim(X[:, 0].min() - 0.5, X[:, 0].max() + 0.5)
ax.set_ylim(X[:, 1].min() - 0.5, X[:, 1].max() + 0.5)
plt.show()

#### 1. The following cell trains Naïve Bayes (NB) and Logistic Regression (LR) classifiers on the train data. Observe the decision boundaries in the figures.

In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import numpy as np
from google.colab import widgets


grid = widgets.Grid(1, 2)


Xtr = X[::2]
Ytr = Y[::2]
Xte = X[1::2]
Yte = Y[1::2]

LR_classifier = LogisticRegression(solver='lbfgs')
LR_classifier.fit(Xtr, Ytr)

NB_classifier = GaussianNB()
NB_classifier.fit(Xtr, Ytr)

# Now to plot the decision regions:

with grid.output_to(0, 0):
  # first the logistic regression classifier:
  fig = plt.figure(1, figsize=(7, 7))
  # fig, axs = plt.subplots(1, 2)
  ax = fig.add_subplot(111)
  ax.set_aspect('equal')

  x_min = min(Xtr[:, 0].min(), Xte[:, 0].min()) - .5 # min of the x axis
  x_max = max(Xtr[:, 0].max(), Xte[:, 0].max()) + .5 # max of the x axis
  y_min = min(Xtr[:, 1].min(), Xte[:, 1].min()) - .5 # min of the y axis
  y_max = max(Xtr[:, 1].max(), Xte[:, 1].max()) + .5 # max of the y axis
  h = .02  # step size in the mesh
  xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))


  Z = LR_classifier.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)
  plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Pastel2)

  # plotting the train points with reduced opacity:
  marker_list = ['o', 'x']
  for l in [1, 2]:
    plt.scatter(Xtr[Ytr == l, 0], Xtr[Ytr == l, 1], 
                marker=marker_list[l-1], color=colors[l-1], s=70, alpha=0.3,)
  # plotting the test points in full opacity:
  for l in [1, 2]:
    plt.scatter(Xte[Yte == l, 0], Xte[Yte == l, 1], 
                marker=marker_list[l-1], color=colors[l-1], s=70, 
                label='{:d} ({:s})'.format(l, iris.target_names[l]))

  ax.set_title('Decision Boundary for Logistic Regression', fontsize=16)
  ax.set_xlabel(iris.feature_names[0], fontsize=14)
  ax.set_ylabel(iris.feature_names[1], fontsize=14)
  ax.set_xlim(xx.min(), xx.max())
  ax.set_ylim(yy.min(), yy.max())
  ax.legend(fontsize=12, loc='upper left')
  ax.grid(alpha=0.3)
  plt.show()

with grid.output_to(0, 1):
  # Now the Naive Bayes classifier:
  fig = plt.figure(1, figsize=(7, 7))
  ax = fig.add_subplot(111)
  ax.set_aspect('equal')

  Z = NB_classifier.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)
  plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Pastel2)

  # plotting the train points with reduced opacity:
  marker_list = ['o', 'x']
  for l in [1, 2]:
    plt.scatter(Xtr[Ytr == l, 0], Xtr[Ytr == l, 1], 
                marker=marker_list[l-1], color=colors[l-1], s=70, alpha=0.3,)
  # plotting the test points in full opacity:
  for l in [1, 2]:
    plt.scatter(Xte[Yte == l, 0], Xte[Yte == l, 1], 
                marker=marker_list[l-1], color=colors[l-1], s=70, 
                label='{:d} ({:s})'.format(l, iris.target_names[l]))

  ax.set_title('Decision Boundary for Naive Bayes', fontsize=16)
  ax.set_xlabel(iris.feature_names[0], fontsize=14)
  ax.set_ylabel(iris.feature_names[1], fontsize=14)
  ax.set_xlim(xx.min(), xx.max())
  ax.set_ylim(yy.min(), yy.max())
  ax.legend(fontsize=12, loc='upper left')
  ax.grid(alpha=0.3)
  plt.show()

#### 2. Typically we make decisions based on which class has greater probability (i.e., which class has > 0.5 probability in the binary case). However, sometimes it is more important to detect one class than the other. In such cases we can tune the threshold to prefer one class or the other. In the following code, you can set the threshold value. Try some threshold values between 0 and 1. Observe how the decision boundaries visually change in the two figures, as well as the displayed confusion matrices.

In [0]:
from sklearn.metrics import confusion_matrix


threshold = 0.7


grid = widgets.Grid(1, 2)


with grid.output_to(0, 0):
  # first for LR classifier:
  predict_index = LR_classifier.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,0]>threshold
  Z = np.ones(predict_index.shape, dtype=int)
  Z[~predict_index] = 2
  Z = Z.reshape(xx.shape)


  fig = plt.figure(1, figsize=(7, 7))
  ax = fig.add_subplot(111)
  ax.set_aspect('equal')

  plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Pastel2)

  # plotting the train points with reduced opacity:
  marker_list = ['o', 'x']
  for l in [1, 2]:
    plt.scatter(Xtr[Ytr == l, 0], Xtr[Ytr == l, 1], 
                marker=marker_list[l-1], color=colors[l-1], s=70, alpha=0.3,)
  # plotting the test points in full opacity:
  for l in [1, 2]:
    plt.scatter(Xte[Yte == l, 0], Xte[Yte == l, 1], 
                marker=marker_list[l-1], color=colors[l-1], s=70, 
                label='{:d} ({:s})'.format(l, iris.target_names[l]))

  ax.set_title('Decision Boundary for Logistic Regression with threshold={}'.format(threshold), fontsize=16)
  ax.set_xlabel(iris.feature_names[0], fontsize=14)
  ax.set_ylabel(iris.feature_names[1], fontsize=14)
  ax.set_xlim(xx.min(), xx.max())
  ax.set_ylim(yy.min(), yy.max())
  ax.legend(fontsize=12, loc='upper left')
  ax.grid(alpha=0.3)
  plt.show()
  
  predict_index = LR_classifier.predict_proba(Xte)[:,0]>threshold
  Yte_pred = np.ones(predict_index.shape, dtype=int)
  Yte_pred[~predict_index] = 2

  print('Confusion matrix (on test data):')
  print(confusion_matrix(Yte, Yte_pred))




with grid.output_to(0, 1):
  # Now the Naive Bayes classifier:
  fig = plt.figure(1, figsize=(7, 7))
  ax = fig.add_subplot(111)
  ax.set_aspect('equal')

  predict_index = NB_classifier.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,0]>threshold
  Z = np.ones(predict_index.shape, dtype=int)
  Z[~predict_index] = 2
  Z = Z.reshape(xx.shape)
  plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Pastel2)

  # plotting the train points with reduced opacity:
  marker_list = ['o', 'x']
  for l in [1, 2]:
    plt.scatter(Xtr[Ytr == l, 0], Xtr[Ytr == l, 1], 
                marker=marker_list[l-1], color=colors[l-1], s=70, alpha=0.3,)
  # plotting the test points in full opacity:
  for l in [1, 2]:
    plt.scatter(Xte[Yte == l, 0], Xte[Yte == l, 1], 
                marker=marker_list[l-1], color=colors[l-1], s=70, 
                label='{:d} ({:s})'.format(l, iris.target_names[l]))

  ax.set_title('Decision Boundary for Naive Bayes with threshold={}'.format(threshold), fontsize=16)
  ax.set_xlabel(iris.feature_names[0], fontsize=14)
  ax.set_ylabel(iris.feature_names[1], fontsize=14)
  ax.set_xlim(xx.min(), xx.max())
  ax.set_ylim(yy.min(), yy.max())
  ax.legend(fontsize=12, loc='upper left')
  ax.grid(alpha=0.3)
  plt.show()

  predict_index = NB_classifier.predict_proba(Xte)[:,0]>threshold
  Yte_pred = np.ones(predict_index.shape, dtype=int)
  Yte_pred[~predict_index] = 2
  print('Confusion matrix (on test data):')
  print(confusion_matrix(Yte, Yte_pred))


#### 3. The threshold to use provides a parameter for the user of the classifier to control. In this way they can for example maintain a maximum false positive rate, or a minimum true positive rate according to the application. 

One way to evaluate a classifier so its suitability for any application can be checked is to calculate true positive rate (TPR) and false positive rate (FPR) for the entire range of thresholds.


---
#### <font color='maroon'>**Exercise 3:** The following cell contains a function that given a classifier and a threshold and some (test) samples, returns the TPR and FPR. Use this functoin to try a bunch of thresholds and fill in the TPR and FPR vectors. Plot TPR (y-axis) against FPR (x-axis) to visualise the resulting ROC curve. Provide both your code and the figure. [1 mark] </ins></font>
--- 

In [0]:
def compute_tpr_fpr(classifier, threshold, Xte, Yte):
  classes = np.unique(Yte)
  predict_index = classifier.predict_proba(Xte)[:,0]>threshold
  true_positives = sum((Yte == classes[0])&(predict_index))
  false_positives = sum((Yte == classes[1])&(predict_index))
  all_actual_positives = sum(Yte == classes[0])
  all_actual_negatives = sum(Yte == classes[1])
  
  TPR = true_positives/all_actual_positives
  FPR = false_positives/all_actual_negatives
  return TPR, FPR

# just for testing the function:
threshold = 0.5
TPR, FPR = compute_tpr_fpr(LR_classifier, threshold, Xte, Yte)
print(TPR, FPR)

In [0]:
# Now, write your code here (to compute a 1-d arrays of TPR and FRP for different 
# values of the threshold)





In [0]:
# Here is a sample plotting code, once you get the arrays of TPR and FPR for 
# each of the classifiers


plt.figure(figsize=(8,8))
plt.title('ROC Curves')
plt.plot([0, 1], [0, 1], 'k--')
plt.axis([-0.005, 1, 0, 1.005])
plt.xticks(np.arange(0,1, 0.05), rotation=90)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
    

plt.plot(FPR_LR, TPR_LR, linewidth=3, alpha=0.7, label='Logistic Regression')

plt.plot(FPR_NB, TPR_NB, linewidth=3, alpha=0.7, label='Naive Bayes')

plt.legend(loc='best', fontsize=14)

From the ROC curve, you can see that one classifier or the other may be
slightly preferable depending on the required TPR/FPR constraints of the
particular application. The Area under the ROC curve is the preferred metric to compare two curves independently of a specific choice of threshold. 

---
#### <font color='maroon'>**Exercise 4:** Compare the AUC of the ROCs of the two classifiers. Which one is preferable by the AUC metric? [1 mark] </ins></font>
--- 

In [0]:
# your code here to compute the AUC (you don't have to include the code in your answer!)



---
#### <font color='maroon'>**Exercise 5:** Suppose for a particular application, the maximum allowed FPR is 0.16.  Which classifier is preferable? Obtains the maximum TPR given this FPR constraint? [1 mark] </ins></font>
--- 

In [0]:
# your code here (again, you don't need to include your code for the answer!)